<a href="https://colab.research.google.com/github/belindadmr/AnalisisK-NN/blob/main/Analisis_K_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- Nama : Belinda Damara Datu
- NIM : 4112321016
- Mata Kuliah : Data Mining
- Prodi : Statistika Terapan dan Komputasi

# Analisis K-NN

Jarak Mahalanobis adalah metode untuk mengukur jarak antara titik dan suatu distribusi dalam ruang berdimensi banyak. Jarak ini mengambil pertimbangan kovariansi antar variabel, oleh karena itu, ia dapat mengukur seberapa jauh suatu titik berada dari distribusi dengan memperhitungkan struktur variabilitas dari data

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import DistanceMetric
from sklearn.neighbors import KNeighborsClassifier
from scipy.spatial import distance
from sklearn.metrics import accuracy_score

In [ ]:
pip install pandas scikit-learn numpy

In [ ]:
!unzip wine.zip

Archive:  wine.zip
  inflating: Index                   
  inflating: wine.data               
  inflating: wine.names              


In [ ]:
column_names = [
    'Class', 'Alcohol', 'Malic acid', 'Ash', 'Alcalinity of ash', 'Magnesium',
    'Total phenols', 'Flavanoids', 'Nonflavanoid phenols', 'Proanthocyanins',
    'Color intensity', 'Hue', 'OD280/OD315 of diluted wines', 'Proline'
]

df = pd.read_csv('wine.data', header=None, names=column_names)


In [ ]:
print(df.describe())

            Class     Alcohol  Malic acid         Ash  Alcalinity of ash  \
count  178.000000  178.000000  178.000000  178.000000         178.000000   
mean     1.938202   13.000618    2.336348    2.366517          19.494944   
std      0.775035    0.811827    1.117146    0.274344           3.339564   
min      1.000000   11.030000    0.740000    1.360000          10.600000   
25%      1.000000   12.362500    1.602500    2.210000          17.200000   
50%      2.000000   13.050000    1.865000    2.360000          19.500000   
75%      3.000000   13.677500    3.082500    2.557500          21.500000   
max      3.000000   14.830000    5.800000    3.230000          30.000000   

        Magnesium  Total phenols  Flavanoids  Nonflavanoid phenols  \
count  178.000000     178.000000  178.000000            178.000000   
mean    99.741573       2.295112    2.029270              0.361854   
std     14.282484       0.625851    0.998859              0.124453   
min     70.000000       0.980000   

In [ ]:
print(df.head())

   Class  Alcohol  Malic acid   Ash  Alcalinity of ash  Magnesium  \
0      1    14.23        1.71  2.43               15.6        127   
1      1    13.20        1.78  2.14               11.2        100   
2      1    13.16        2.36  2.67               18.6        101   
3      1    14.37        1.95  2.50               16.8        113   
4      1    13.24        2.59  2.87               21.0        118   

   Total phenols  Flavanoids  Nonflavanoid phenols  Proanthocyanins  \
0           2.80        3.06                  0.28             2.29   
1           2.65        2.76                  0.26             1.28   
2           2.80        3.24                  0.30             2.81   
3           3.85        3.49                  0.24             2.18   
4           2.80        2.69                  0.39             1.82   

   Color intensity   Hue  OD280/OD315 of diluted wines  Proline  
0             5.64  1.04                          3.92     1065  
1             4.38  1.05  

In [ ]:
# Menghilangkan kolom 'Class' karena fokus pada fitur
df_features = df.drop(columns='Class')

# Hitung vektor rata-rata dan matriks kovariansi
mean_vector = df_features.mean().values
cov_matrix = df_features.cov().values
inv_cov_matrix = np.linalg.inv(cov_matrix)

# Fungsi untuk menghitung jarak Mahalanobis
def mahalanobis(x, mean_vector, inv_cov_matrix):
    delta = x - mean_vector
    return np.sqrt(delta.T @ inv_cov_matrix @ delta)

# Hitung jarak Mahalanobis untuk setiap titik dalam dataset
mahalanobis_distances = df_features.apply(mahalanobis, axis=1, args=(mean_vector, inv_cov_matrix))


In [ ]:
print(mahalanobis_distances)

0      3.567329
1      3.131736
2      3.064510
3      3.520709
4      2.584803
         ...   
173    3.226328
174    2.258621
175    3.525690
176    3.187978
177    3.322199
Length: 178, dtype: float64


In [ ]:
distances_list = [3.567329, 3.131736, 3.064510, 3.520709, 2.584803, ...]
mahalanobis_distances = pd.Series(distances_list)

print(mahalanobis_distances)

0    3.567329
1    3.131736
2     3.06451
3    3.520709
4    2.584803
5    Ellipsis
dtype: object


In [ ]:
print(df.head())

   Class  Alcohol  Malic acid   Ash  Alcalinity of ash  Magnesium  \
0      1    14.23        1.71  2.43               15.6        127   
1      1    13.20        1.78  2.14               11.2        100   
2      1    13.16        2.36  2.67               18.6        101   
3      1    14.37        1.95  2.50               16.8        113   
4      1    13.24        2.59  2.87               21.0        118   

   Total phenols  Flavanoids  Nonflavanoid phenols  Proanthocyanins  \
0           2.80        3.06                  0.28             2.29   
1           2.65        2.76                  0.26             1.28   
2           2.80        3.24                  0.30             2.81   
3           3.85        3.49                  0.24             2.18   
4           2.80        2.69                  0.39             1.82   

   Color intensity   Hue  OD280/OD315 of diluted wines  Proline  
0             5.64  1.04                          3.92     1065  
1             4.38  1.05  

In [ ]:
# Membagi data menjadi training set dan testing set
X = df.drop('Class', axis=1)
y = df['Class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Mendefinisikan fungsi jarak Mahalanobis
def mahalanobis_dist(x, y, VI):
    return distance.mahalanobis(x, y, VI)

# Menghitung matriks kovariansi dari data training
cov_matrix = np.cov(X_train, rowvar=False)
inv_cov_matrix = np.linalg.pinv(cov_matrix)

# Menerapkan k-NN dengan jarak Mahalanobis
results = {}
for k in [2, 3, 4, 5, 6, 7]:
    if y_test.iloc[0] % 2 == 0 and k in [3, 5, 7] or y_test.iloc[0] % 2 != 0 and k in [2, 4, 6]:
        knn = KNeighborsClassifier(n_neighbors=k, metric=mahalanobis_dist, metric_params={'VI': inv_cov_matrix})
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        acc = accuracy_score(y_test, y_pred)
        results[k] = acc

print(results)


{2: 0.8611111111111112, 4: 0.8888888888888888, 6: 0.9166666666666666}
